In [2]:
import requests
import pandas as pd
import json
import pickle

## Pruebas de API

- Definimos un diccionario de prueba.

In [3]:
datos = {
    'Age': float(30),
    'BusinessTravel': str('Travel_Rarely'),
    'Department': str('Sales'),
    'DistanceFromHome': float(1),
    'Education': str('Below College'),
    'EducationField': str('Life Sciences'),
    'Gender': str('Female'),
    'JobLevel': str(1),
    'JobRole': str('Healthcare Representative'),
    'MaritalStatus': str('Married'),
    'MonthlyIncome': float(300000),
    'NumCompaniesWorked': float(1),
    'PercentSalaryHike': float(11),
    'StockOptionLevel': str(0),
    'TotalWorkingYears': float(1),
    'TrainingTimesLastYear': str(1),
    'YearsAtCompany': float(1),
    'YearsSinceLastPromotion': float(1),
    'YearsWithCurrManager': float(1),
    'EnvironmentSatisfaction': str(1),
    'JobSatisfaction': str(1),
    'WorkLifeBalance': str(1),
    'JobInvolvement': str(1),
    'PerformanceRating': str(3),
}



- Lo convertimos en dataframe.

In [6]:
df_consulta = pd.DataFrame(datos, index=[0])
df_consulta

,Age,BusinessTravel,Department,DistanceFromHome,Education,EducationField,Gender,JobLevel,JobRole,MaritalStatus,...,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,JobInvolvement,PerformanceRating
0,30.0,Travel_Rarely,Sales,1.0,Below College,Life Sciences,Female,1,Healthcare Representative,Married,...,1.0,1,1.0,1.0,1.0,1,1,1,1,3


- Y en diccionario, dado que es el formato en el que las APIs envían y reciben información.

In [ ]:
json_consulta = json.dumps(df_consulta.iloc[0].to_dict())

- Usamos 'post' para enviar información a la API (nuestro diccionario con los datos para la predicción).

In [209]:
res = requests.post("http://127.0.0.1:5000/predict", json= datos)

- Y vemos que recibimos una respuesta, que es son los valores de la predicción (riesgo y probabilidad).

In [211]:
res.json()

{'attrition_risk': 'No', 'probability': 0.0}

## Pruebas de encoder, scaler y modelo

- Cargamos los elementos generados como archivos pickle y creamos un dataframe con el diccionario de 'datos'.

In [13]:
# Cargar los archivos pickle (encoder, scaler y modelo)
with open('../transformers/encoder.pkl', 'rb') as file:
    encoder = pickle.load(file)
    print("Encoder cargado correctamente.")

with open('../transformers/scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)
    print("Scaler cargado correctamente.")

with open('../transformers/model.pkl', 'rb') as file:
    model = pickle.load(file)
    print("Modelo cargado correctamente.")

Encoder cargado correctamente.
Scaler cargado correctamente.
Modelo cargado correctamente.


In [14]:
df = pd.DataFrame([datos])
df

,Age,BusinessTravel,Department,DistanceFromHome,Education,EducationField,Gender,JobLevel,JobRole,MaritalStatus,...,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,JobInvolvement,PerformanceRating
0,30.0,Travel_Rarely,Sales,1.0,Below College,Life Sciences,Female,1,Healthcare Representative,Married,...,1.0,1,1.0,1.0,1.0,1,1,1,1,3


- Vamos a comprobar también que el encoder y el scaler contengan las columnas correctas.

In [15]:
encoder.cols

['Gender',
 'Education',
 'JobLevel',
 'StockOptionLevel',
 'PerformanceRating',
 'BusinessTravel',
 'Department',
 'EducationField',
 'JobRole',
 'MaritalStatus',
 'TrainingTimesLastYear',
 'EnvironmentSatisfaction',
 'JobSatisfaction',
 'WorkLifeBalance',
 'JobInvolvement']

In [16]:
scaler.get_feature_names_out()

array(['Age', 'DistanceFromHome', 'MonthlyIncome', 'NumCompaniesWorked',
       'PercentSalaryHike', 'TotalWorkingYears', 'YearsAtCompany',
       'YearsSinceLastPromotion', 'YearsWithCurrManager'], dtype=object)

- Creamos una columna 'dummy' en el DF, dado que el objeto del encoder espera recibir 25 columnas, y aquí tenemos solamente 24, siendo la 25 la predicción de la varible respuesta ('Attrition').

- Aplicamos el encoder y eliminamos esa columna, comprobando el DF.

In [8]:
df['Attrition'] = 0  # valor temporal para el encoding
df_encoded = encoder.transform(df)
df_encoded.drop(columns='Attrition', inplace= True)

In [9]:
df_encoded

,Age,BusinessTravel,Department,DistanceFromHome,Education,EducationField,Gender,JobLevel,JobRole,MaritalStatus,...,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,JobInvolvement,PerformanceRating
0,30.0,0.145815,0.147368,1.0,0.159847,0.164869,0.15235,0.147766,0.144828,0.117483,...,1.0,0.146667,1.0,1.0,1.0,0.159847,0.159847,0.159847,0.215909,0.156863


- Aplicaremos ahora nuestro escalador a las columnas numéricas. Contrario al encoder, aquí debemos indicarle cuáles son las columnas que debe transformar.

- Comprobamos que tenemos nuestro DF codificado y estandarizado.

In [ ]:
col_num=['Age', 'DistanceFromHome', 'MonthlyIncome', 'NumCompaniesWorked',
    'PercentSalaryHike', 'TotalWorkingYears', 'YearsAtCompany',
    'YearsSinceLastPromotion', 'YearsWithCurrManager']

df_encoded[col_num] = scaler.transform(df_encoded[col_num])

In [11]:
df_encoded

,Age,BusinessTravel,Department,DistanceFromHome,Education,EducationField,Gender,JobLevel,JobRole,MaritalStatus,...,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,EnvironmentSatisfaction,JobSatisfaction,WorkLifeBalance,JobInvolvement,PerformanceRating
0,-0.747889,0.145815,0.147368,-1.003804,0.159847,0.164869,0.15235,0.147766,0.144828,0.117483,...,-1.339221,0.146667,-1.002783,-0.361313,-0.870216,0.159847,0.159847,0.159847,0.215909,0.156863


- Con este DF, podremos realizar la predicción, siendo 0 "No" y 1 "Sí".

In [12]:
prediccion = model.predict(df_encoded)
probabilidad = model.predict_proba(df_encoded)[0, 1]
print("Predicción:", prediccion, "Probabilidad:", probabilidad)

Predicción: [0.] Probabilidad: 0.0
